# Foreshock–Aftershock Evaluation (SeisLM-style)

This notebook loads a fine-tuned checkpoint, runs evaluation on the foreshock–aftershock test split, and plots a confusion matrix.

In [41]:
# Setup: adjust working directory and imports
import os, sys
base_dir = os.path.abspath(os.path.join(os.getcwd()))
if os.path.basename(base_dir) != 'seismic_data_modeling':
    # Try to locate project root ending with 'seismic_data_modeling'
    parts = base_dir.split(os.sep)
    if 'seismic_data_modeling' in parts:
        idx = parts.index('seismic_data_modeling')
        proj = os.sep.join(parts[:idx+1])
        os.chdir(proj)
        base_dir = proj

sys.path.insert(0, base_dir)
print('Project dir:', base_dir)

import torch
import torch.nn.functional as F
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt

from dataloaders.foreshock_aftershock_lit import ForeshockAftershockLitDataset
from simple_train import load_checkpoint


Project dir: /scicore/home/dokman0000/alvani0000/final_seismology/seismic_data_modeling


**Note:** The pretraining (contrastive) checkpoint initializes the backbone before finetuning.
For evaluation here, we load the finetuned foreshock checkpoint only.

- Pretraining ckpt (for finetune init):
  `/scicore/home/dokman0000/alvani0000/final_seismology/wandb_logs/mars/contrastive_resume_58609395.ckpt/checkpoints/callback-epoch=38-step=592102.ckpt`
- Foreshock finetuned ckpt (for eval):
  `/scicore/home/dokman0000/alvani0000/final_seismology/seismic_data_modeling/wandb_logs/mars/2025-10-19__19_08_46__j59201230/checkpoints/callback-epoch=4-step=874.ckpt`


In [42]:
# Paths and config
DATA_DIR = '/scicore/home/dokman0000/alvani0000/seis_data'
# Use FINETUNED foreshock checkpoint for evaluation (not the pretraining ckpt)
CKPT_PATH = '/scicore/home/dokman0000/alvani0000/final_seismology/seismic_data_modeling/wandb_logs/mars/2025-10-19__19_08_46__j59201230/checkpoints/callback-epoch=4-step=874.ckpt'
NUM_CLASSES = 9
BATCH_SIZE = 32  # evaluation batch size

print('Using DATA_DIR:', DATA_DIR)
print('Using FINETUNED CKPT_PATH:', CKPT_PATH)


Using DATA_DIR: /scicore/home/dokman0000/alvani0000/seis_data
Using FINETUNED CKPT_PATH: /scicore/home/dokman0000/alvani0000/final_seismology/seismic_data_modeling/wandb_logs/mars/2025-10-19__19_08_46__j59201230/checkpoints/callback-epoch=4-step=874.ckpt


In [43]:
# Build test dataloader (mirrors training pipeline)
ds = ForeshockAftershockLitDataset(
    data_dir=DATA_DIR,
    num_classes=NUM_CLASSES,
    batch_size=BATCH_SIZE,
    event_split_method='temporal',
    component_order='ZNE',
    seed=42,
    remove_class_overlapping_dates=False,
    train_frac=0.7,
    val_frac=0.10,
    test_frac=0.20,
    dimension_order='NWC',  # match training setup
    demean_axis=1,
    amp_norm_axis=1,
    amp_norm_type='std',
    num_workers=0,
    collator=None,
)
test_loader = ds.test_loader
len(test_loader)


Seed set to 42
Seed set to 42


50

In [44]:
# Load model from finetuned checkpoint using Hydra-composed config
from hydra import initialize, compose
from simple_train import load_checkpoint
# Compose full config with defaults resolved
with initialize(config_path='configs', version_base=None):
    cfg = compose(config_name='config.yaml', overrides=[
        'experiment=fore_aftershock/finetune_xlstm_unet',
        'train.disable_pretraining=true',
    ])
model, _ = load_checkpoint(CKPT_PATH, updated_model_config=cfg, d_data=3)
# Ensure classification
try:
    model.model.pretraining=False
    model.encoder.pretraining=False
except Exception:
    pass
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device).eval()
print('Model loaded via Hydra-composed config on', device)


MissingConfigException: Primary config directory not found.
Check that the config directory '/scicore/home/dokman0000/alvani0000/final_seismology/seismic_data_modeling/notebooks/configs' exists and readable

In [ ]:
# Evaluate: collect predictions and compute loss/accuracy (with basic debug)
all_preds = []
all_targets = []
total_loss = 0.0
n_samples = 0

with torch.no_grad():
    for batch_idx, (x, y) in enumerate(tqdm(test_loader, desc='Testing')):
        x = x.to(device)
        y = y.to(device)
        try:
            logits, targets = model.forward((x, y), batch_idx)
        except Exception as e:
            print('[error] forward failed at batch', batch_idx, e)
            print('x shape:', tuple(x.shape), 'y shape:', tuple(y.shape))
            raise
        loss = F.cross_entropy(logits, targets)
        total_loss += loss.item() * targets.shape[0]
        n_samples += targets.shape[0]
        preds = torch.argmax(logits, dim=1)
        all_preds.append(preds.detach().cpu().numpy())
        all_targets.append(targets.detach().cpu().numpy())

all_preds = np.concatenate(all_preds, axis=0)
all_targets = np.concatenate(all_targets, axis=0)
test_loss = total_loss / max(1, n_samples)
test_acc = accuracy_score(all_targets, all_preds)
print(f'Test Loss: {test_loss:.4f}  |  Test Acc: {test_acc:.4f}')


In [ ]:
# Confusion Matrix
cm = confusion_matrix(all_targets, all_preds, labels=list(range(NUM_CLASSES)))
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(range(NUM_CLASSES)))
disp.plot(ax=ax, cmap='Blues', colorbar=True)
plt.title('Foreshock–Aftershock Confusion Matrix')
plt.tight_layout()
plt.show()
